In [ ]:
import time
import requests
import pandas as pd
import psycopg2
import logging
import os
from datetime import datetime
from io import StringIO
from concurrent.futures import ThreadPoolExecutor

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("data_collection.log"),
        logging.StreamHandler()
    ]
)

# Ensure directory exists
os.makedirs("historical_data", exist_ok=True)

# Database connection parameters
DB_PARAMS = {
    'dbname': 'Stock',
    'user': 'adity',
    'password': 'qwertypoi',
    'host': 'localhost'
}

# Session for NSE requests
session = requests.Session()

# Headers for NSE requests
head = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.nseindia.com/",
}

def get_db_connection():
    """Establish a connection to the PostgreSQL database"""
    try:
        conn = psycopg2.connect(**DB_PARAMS)
        return conn
    except Exception as e:
        logging.error(f"Database connection error: {e}")
        return None

def fetch_stock_symbols():
    """Retrieve all stock symbols from the database"""
    conn = get_db_connection()
    if not conn:
        return []
    
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT symbol FROM companies")
        symbols = [row[0] for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        logging.info(f"Retrieved {len(symbols)} stock symbols from database")
        return symbols
    except Exception as e:
        logging.error(f"Error fetching symbols: {e}")
        if conn:
            conn.close()
        return []

def getHistoryData(company, from_date=None, to_date=None, save_csv=True):
    """Fetch historical data for a given company from NSE"""
    if from_date is None:
        from_date = datetime.today().strftime("%d-%m-%Y")
    if to_date is None:
        to_date = (datetime(datetime.today().year - 1, datetime.today().month, datetime.today().day).strftime("%d-%m-%Y"))

    try:
        # Initialize session with cookies
        session.get("https://www.nseindia.com", headers=head)
        session.get(f"https://www.nseindia.com/get-quotes/equity?symbol={company}", headers=head)
        session.get(f"https://www.nseindia.com/api/historical/cm/equity?symbol={company}", headers=head)

        # Fetch the data
        url = f"https://www.nseindia.com/api/historical/cm/equity?symbol={company}&series=[%22EQ%22]&from={to_date}&to={from_date}&csv=true"
        webdata = session.get(url, headers=head)

        if webdata.status_code != 200:
            logging.error(f"Failed to fetch data for {company}. Status Code: {webdata.status_code}")
            return None

        # Parse CSV data
        df = pd.read_csv(StringIO(webdata.text[3:]))  # Skipping the first three characters
        
        if save_csv:
            filename = f"historical_data/{company}.csv"
            df.to_csv(filename, index=False)
            logging.info(f"✅ Data saved: {filename}")
        
        return df
    except Exception as e:
        logging.error(f"❌ Error fetching data for {company}: {e}")
        return None

def process_symbol(symbol, from_date, to_date):
    """Process a single symbol to get its historical data"""
    try:
        logging.info(f"Processing symbol: {symbol}")
        data = getHistoryData(symbol, from_date, to_date)
        if data is not None:
            logging.info(f"Successfully retrieved data for {symbol} with {len(data)} records")
            return symbol, True
        return symbol, False
    except Exception as e:
        logging.error(f"Failed to process {symbol}: {e}")
        return symbol, False

def collect_all_historical_data(from_date=None, to_date=None, parallel=True, max_workers=5):
    """Collect historical data for all companies"""
    symbols = fetch_stock_symbols()
    
    if not symbols:
        logging.error("No symbols retrieved from the database")
        return
    
    successful = []
    failed = []
    
    if parallel:
        # Use thread pool for parallel processing
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(executor.map(
                lambda symbol: process_symbol(symbol, from_date, to_date), 
                symbols
            ))
        
        # Parse results
        successful = [symbol for symbol, success in results if success]
        failed = [symbol for symbol, success in results if not success]
    else:
        # Sequential processing
        for symbol in symbols:
            result = process_symbol(symbol, from_date, to_date)
            if result[1]:  # If successful
                successful.append(result[0])
            else:
                failed.append(result[0])
            
            # Add a small delay to avoid overloading the server
            time.sleep(1)
    
    # Report results
    logging.info(f"Data collection complete. Successful: {len(successful)}, Failed: {len(failed)}")
    if failed:
        logging.warning(f"Failed symbols: {failed}")
    
    return successful, failed

def save_results_to_db(successful, failed, from_date, to_date):
    """Save collection results to database for tracking"""
    conn = get_db_connection()
    if not conn:
        return False
    
    try:
        cursor = conn.cursor()
        
        # Create table if it doesn't exist
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS collection_results (
            id SERIAL PRIMARY KEY,
            collection_date TIMESTAMP,
            from_date DATE,
            to_date DATE,
            successful_count INTEGER,
            failed_count INTEGER,
            successful_symbols TEXT[],
            failed_symbols TEXT[]
        )
        """)
        
        # Insert results
        cursor.execute("""
        INSERT INTO collection_results 
        (collection_date, from_date, to_date, successful_count, failed_count, successful_symbols, failed_symbols)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            datetime.now(),
            datetime.strptime(from_date, "%d-%m-%Y").date() if from_date else None,
            datetime.strptime(to_date, "%d-%m-%Y").date() if to_date else None,
            len(successful),
            len(failed),
            successful,
            failed
        ))
        
        conn.commit()
        cursor.close()
        conn.close()
        logging.info("Collection results saved to database")
        return True
        
    except Exception as e:
        logging.error(f"Error saving results to database: {e}")
        if conn:
            conn.close()
        return False

if __name__ == "__main__":
    # Default dates - today to 1 year ago
    from_date = datetime.today().strftime("%d-%m-%Y")
    to_date = datetime(datetime.today().year - 1, datetime.today().month, datetime.today().day).strftime("%d-%m-%Y")
    
    # You can override these dates here
    # from_date = "14-04-2025"
    # to_date = "14-04-2024"
    
    logging.info(f"Starting data collection from {to_date} to {from_date}")
    
    # Collect data (using parallel processing by default)
    successful, failed = collect_all_historical_data(
        from_date=from_date,
        to_date=to_date,
        parallel=True,
        max_workers=5  # Adjust based on your system and to avoid rate limiting
    )
        
    logging.info("Script execution completed")

2025-04-14 16:27:37,507 - INFO - Starting data collection from 14-04-2024 to 14-04-2025
2025-04-14 16:27:37,545 - INFO - Retrieved 2466 stock symbols from database
2025-04-14 16:27:37,546 - INFO - Processing symbol: 20MICRONS
2025-04-14 16:27:37,548 - INFO - Processing symbol: 21STCENMGM
2025-04-14 16:27:37,550 - INFO - Processing symbol: 360ONE
2025-04-14 16:27:37,550 - INFO - Processing symbol: 3IINFOLTD
2025-04-14 16:27:37,550 - INFO - Processing symbol: 3MINDIA
2025-04-14 16:27:38,481 - INFO - ✅ Data saved: historical_data/3IINFOLTD.csv
2025-04-14 16:27:38,482 - INFO - Successfully retrieved data for 3IINFOLTD with 248 records
2025-04-14 16:27:38,482 - INFO - Processing symbol: 3PLAND
2025-04-14 16:27:38,488 - INFO - ✅ Data saved: historical_data/360ONE.csv
2025-04-14 16:27:38,489 - INFO - Successfully retrieved data for 360ONE with 248 records
2025-04-14 16:27:38,491 - INFO - Processing symbol: 5PAISA
2025-04-14 16:27:38,508 - INFO - ✅ Data saved: historical_data/3MINDIA.csv
2025-